<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 657: Microwave Remote Sensing <b> </font>

<font size="5"> <b>Lab 6: Volcano Source Modeling Using InSAR <font color='rgba(200,0,0,0.2)'> -- [20 Points] </font> </b> </font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks </b> <br>
<font color='rgba(200,0,0,0.2)'> <b>Due Date: </b> April 09, 2019 </font>
</font>

<img style="padding: 7px" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /> <font size="3"> This lab will introduce you to the intersection between Radar Remote Sensing and Inverse Modeling. Radar Remote Sensing can provide us with geodetic observations of surface deformation. Inverse Modeling helps us understand the physical causes behind an observed deformation. 
    
To illuminate the handoff from geodesy to geophysics, this lab will show how to use InSAR observations to determine the most likely parameters of a volcanic magma source underneath Okmok volcano, Alaska. We will use a Mogi source model to describe the physics behind deformation at Okmok. We will again use our **Jupyter Notebook** framework implemented within the Amazon Web Services (AWS) cloud to work on this exercise. <br><br>

This Lab is part of the UAF course <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657: Microwave Remote Sensing</a>. It will introduce the following data analysis concepts:

- A Mogi Source Model describing volcanic source geometry and physics
- How to use the "grid search" method to perform a pseudo-inversion of a Mogi source model 
- How to solve for the best fitting source parameters using modeling with InSAR data
</font>
<br>
<font size="4"> <font color='rgba(200,0,0,0.2)'> <b>THIS NOTEBOOK INCLUDES THREE HOMEWORK ASSIGNMENTS.</b></font> Complete all six assignments to achieve full score. </font> <br>
<font size="3"> The homework assignments in this lab are to be completed within this notebook. To submit your homework, please <font color='rgba(200,0,0,0.2)'> download your completed notebook both asf PDF (*.pdf) and as a Notebook file (*.ipynb) and submit them as a ZIP bundle via Blackboard or email (to fjmeyer@alaska.edu) </font>. To download, please select the following options in the main menu of the notebook interface:

<ol type="1">
  <li><font color='rgba(200,0,0,0.2)'> <b> Save your notebook with all of its content</b></font> by selecting <i> File / Save and Checkpoint </i> </li>
  <li><font color='rgba(200,0,0,0.2)'> <b>To export in Notebook format</b></font>, click on <i>File / Download as / Notebook (.ipynb)</i></li>
  <li><font color='rgba(200,0,0,0.2)'> <b>To export in PDF format</b></font>, click on <i>File / Download as / PDF vs LaTeX (.pdf) </i></li>
</ol>

<b>Please rename the notebooks to include your name before submitting it via Blackboard</b>. Contact me at fjmeyer@alaska.edu should you run into any problems.
</font>

</font>

<hr>
<font face="Calibri" size="5" color="red"> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3"> First step in any notebook is to import the required Python libraries into the Jupyter environment. In this notebooks we use the following scientific libraries:
<ol type="1">
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. </li>
</font>
<br>
<font face="Calibri" size="3">The first step is to <b>import all required python modules:</b></font>

In [1]:
# Scientific Libraries
import matplotlib.pylab as plt # for add_subplot, cm.jet, colorbar, figure, grid, imshow, rcParams.update, savefig,
                               # set_bad, set_clim, set_title, set_xlabel, set_ylabel
import numpy as np # for arange, arctan, concatenate, cos, fromfile, isnan, ma.masked_value, min, pi, power, reshape,
                   # sqrt, square, sin, sum, tile, transpose, where, zeros     

# ASF's Jupyter Notebook Module
from asf_notebook import path_exists
from asf_notebook import new_directory


# General Purpose Libraries
import os # for chdir, getcwd, path.basename, path.exists
import subprocess

<font face="Calibri" size="3"><b>set up matplotlib plotting</b> inside the notebook:</font>

In [2]:
%matplotlib inline

<hr>
<font face="Calibri">

<font size="5"> <b> 1. InSAR at Okmok Volcano, Alaska </b> </font>  

<img style="padding: 7px" src="NotebookAddons/Lab6-OkmokdefoGPS.JPG" width="550" align="right" /><font size="3"> Okmok is one of the more active volcanoes in Alaska’s Aleutian Chain. Its last (known) eruption was in the summer of 2008. Okmok is interesting from an InSAR perspective as it inflates and deflates heavily as magma moves around in its magmatic source located roughly 2.5 km underneath the surface. To learn more about Okmok volcano and its eruptive history, please visit the very informative site of the <a href="https://avo.alaska.edu/volcanoes/activity.php?volcname=Okmok&eruptionid=604&page=basic" target="_blank">Alaska Volcano Observatory</a>.

This lab uses a pair of C-band ERS-2 SAR images acquired on Aug 18, 2000 and Jul 19, 2002 to analyze the properties of a volcanic source that was responsible for an inflation of Okmok volcano of more than 3 cm near its summit. The figure to the right shows the Okmok surface deformation as measured by GPS data from field campaigns conducted in 2000 and 2002. The plots show that the deformation measured at the site is consistent with that created by an inflating point (Mogi) source.<br> 

<b>The primary goal of the problem set is to estimate values for four unknown model parameters describing a source process beneath a volcano.</b> The lab uses real InSAR data from Okmok volcano, so you should get some sense for how remote sensing can be used to infer physical processes at volcanoes. We will assume that the source can be modeled as an inflating point source (a so-called Mogi source; see <a href="https://radar.community.uaf.edu/files/2019/03/2019-Lecture14_UsingInSARinGeophysics.pdf" target="_blank">Lecture 14</a>) and will use a grid-search method for finding the source model parameters (3D source location and volume of magma influx) that best describe our InSAR-observed surface deformation.
</font>
</font>

<hr>
<font face="Calibri" size="5"> <b> 2. Download and Plot the Observed Deformation Map </b>  
    
<font face="Calibri" size="4"> <b> 2.1 Download Data from AWS S3 Storage Bucket</b><br>
<font size="3"> We are using a deformation map created from C-band ERS-2 SAR images acquired on Aug 18, 2000 and Jul 19, 2002. The deformation map is <b>available to you on the Class AWS S3 data storage bucket:</b> </font>

<font face="Calibri" size="3"><b>Create and move to a directory in which to store our Lab 6 files:"</font>

In [4]:
new_directory("/home/jovyan/notebooks/ASF/GEOS_657_Labs/lab_6_data")
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

current directory: /home/jovyan/notebooks/ASF/GEOS_657_Labs/lab_6_data


<font face="Calibri" size="3"><b>Download the deformation map from the AWS-S3 bucket:</b></font>

In [ ]:
deformation_map_path = 's3://asf-jupyter-data/E451_20000818_20020719.unw'
deformation_map = os.path.basename(deformation_map_path)
subprocess.check_call(['aws', 's3', 'cp',  deformation_map_path, deformation_map])

<font face="Calibri" size="3"><b>Define some global variables:</b></font>

In [ ]:
SAMPLE = 1100
LINE = 980
POSTING = 40.0
HALF_WAVE = 28.3

<font face="Calibri" size="3"><b>Read the dataset into the notebook</b>, storing our observed deformation map in the variable <i>"observed_deformation_map"</i>: </font>

In [ ]:
if path_exists(deformation_map):
    with open (deformation_map, 'rb') as f:    
        coh = np.fromfile(f, dtype='>f', count=-1)
    observed_deformation_map = np.reshape(coh, (LINE, SAMPLE))

<font face="Calibri" size="3"><b>Change the units to cm and replace all nans with 0</b></font>

In [ ]:
observed_deformation_map = observed_deformation_map*HALF_WAVE/2.0/np.pi
where_are_NaNs = np.isnan(observed_deformation_map)
observed_deformation_map[where_are_NaNs] = 0

<font face="Calibri" size="3"> <b>Create a mask</b> that removes invalid samples (low coherence) from the deformation map: </font>

In [ ]:
observed_deformation_map_m = np.ma.masked_where(observed_deformation_map==0, observed_deformation_map)

<hr>
<font face="Calibri" size="4"> <b> 2.2 Visualize The Deformation Map </b>  

<font size="3"> We will visualize the deformation map both in units of [cm] and as a rewrapped interferogram.</font>
<br><br>
<font size="3"><b>Write a function that calculates the bounding box.</b></font>

In [ ]:
def extents(vector_component):
        delta = vector_component[1] - vector_component[0]
        return [vector_component[0] - delta/2, vector_component[-1] + delta/2]

<font size="3"><b>Write a plotting function</b>:</font>

In [ ]:
def plot_model(infile, line, sample, posting):
    # Calculate the bounding box
    extent_xvec = extents((np.arange(1, sample*posting, posting)) / 1000)
    extent_yvec = extents((np.arange(1, line*posting, posting)) / 1000)
    extent_xy = extent_xvec + extent_yvec
    
    plt.rcParams.update({'font.size': 14})
    inwrapped = (infile/10 + np.pi) % (2*np.pi) - np.pi
    cmap = plt.cm.jet
    cmap.set_bad('white', 1.)
    
    # Plot displacement
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(1, 2, 1)
    im = ax1.imshow(infile, interpolation='nearest', cmap=cmap, extent=extent_xy, origin='upper')
    cbar = ax1.figure.colorbar(im, ax=ax1, orientation='horizontal')
    ax1.set_title("Displacement in look direction [mm]")
    ax1.set_xlabel("Easting [km]")
    ax1.set_ylabel("Northing [km]")
    plt.grid()
    
    # Plot interferogram
    im.set_clim(-30, 30)
    ax2 = fig.add_subplot(1, 2, 2)
    im = ax2.imshow(inwrapped, interpolation='nearest', cmap=cmap, extent=extent_xy, origin='upper')
    cbar = ax2.figure.colorbar(im, ax=ax2, orientation='horizontal')
    ax2.set_title("Interferogram phase [rad]")
    ax2.set_xlabel("Easting [km]")
    ax2.set_ylabel("Northing [km]")
    plt.grid()

<font face="Calibri" size="3">Call plot_model() to <b>plot our observed deformation map:</b> </font>

In [ ]:
plot_model(observed_deformation_map_m, LINE, SAMPLE, POSTING)
plt.savefig('Okmok-inflation-observation.png', dpi=200, transparent='false')

<hr>
<font face="Calibri" size="5"> <b> 3. The Mogi Source Model and InSAR</b>  
    
<font face="Calibri" size="4"> <b> 3.1 The Mogi Equations</b><br>
<font size="3"> The Mogi model provides the 3D ground displacement, $u(x,y,z)$, due to an inflating source at location $(x_s,y_s,z_s)$ with volume change $V$:

\begin{equation}
u(x,y,z)=\frac{1}{\pi}(1-\nu)\cdot V\Big(\frac{x-x_s}{r(x,y,z)^3},\frac{y-y_s}{r(x,y,z)^3},\frac{z-z_s}{r(x,y,z)^3}\Big)
\end{equation}
<br>
\begin{equation}
r(x,y,z)=\sqrt{(x-x_s)^2+(y-y_s)^2+(z-z_s)^2}
\end{equation}

where $r$ is the distance from the Mogi source to $(x,y,z)$, and $\nu$ is the Poisson's ratio of the halfspace. The Poisson ratio describes how rocks react when put under stress (e.g., pressure). It is affected by temperature, the quantity of liquid to solid, and the composition of the soil material. <b>In our problem, we will assume that $\nu$ is fixed</b>. 
 </font>

<hr>
<font face="Calibri" size="4"> <b> 3.2 Projecting Mogi Deformation to InSAR Line-of-Sight</b><br>
<font size="3"> In our example, the $x$-axis points east, $y$ points north, and $z$ points up. However, in the code the input values for $z$ are assumed to be depth, such that the Mogi source is at depth $z_s > 0$. The observed interferogram is already corrected for the effect of topography, so the observations can be considered to be at $z = 0$.
    
<img style="padding: 7px" src="NotebookAddons/Lab6-LOSprojection.JPG" width="650" align="center" />
The satellite “sees” a projection of the 3D ground displacement, $u$, onto the look vector, $\hat{L}$, which points from the satellite to the target. Therefore, we are actually interested in the (signed magnitude of the) projection of $u$ onto $\hat{L}$ (right). This is given by

\begin{array}{lcl} proj_{\hat{L}}u & = & (u^T\hat{L})\hat{L} \\ u^T\hat{L} & = & u \cdot \hat{L} = |u||\hat{L}|cos(\alpha) = |u|cos(\alpha) \\ & = & u_x\hat{L}_x+ u_y\hat{L}_y + u_z\hat{L}_z \end{array}

where the look vector is given by $\hat{L}=(sin(l) \cdot cos(t), -sin(l) \cdot sin(t), -cos(l))$, where $l$ is the look angle measured from the nadir direction and $t$ is the satellite track angle measured clockwise from geographic north. All vectors are represented in an east-north-up basis.

Our forward model takes a Mogi source, $(x_s,y_s,z_s,V)$, and computes the look displacement at any given $(x, y, z)$ point. If we represent the <i>i</i>th point on our surface grid by $x_i = (x_i,y_i,z_i)$ the the displacement vector is $u_i = u(x_i, y_i, z_i)$, and the look displacement is

\begin{equation}
d_i = u_i \cdot \hat{L}
\end{equation}

<br>
<hr>
<font size="4"> <b> 3.3 Defining the Mogi Forward Model</b><br></font>
<font size="3">We can now represent the Mogi <i>forward problem</i> as 

\begin{equation}
g(m) = d
\end{equation}

where $g(·)$ describes the forward model in the very first equation in this notebook, $m$ is the (unknown) Mogi model, and $d$ is the predicted interferogram. The following code cells calculate the Mogi forward model according to the equations given above:

</font>


<font face="Calibri" size="3"><b>Write a function to calculate a forward model for a Mogi source.</b> </font>

In [ ]:
def calc_forward_model_mogi(n1, e1, depth, delta_volume, northing, easting, plook):
    
    # This geophysical coefficient is needed to describe how pressure relates to volume change
    displacement_coefficient = (1e6*delta_volume*3)/(np.pi*4)
    
    # Calculating the horizontal distance from every point in the deformation map to the x/y source location
    d_mat = np.sqrt(np.square(northing-n1) + np.square(easting-e1))
    
    # denominator of displacement field for mogi source
    tmp_hyp = np.power(np.square(d_mat) + np.square(depth),1.5)
    
    # horizontal displacement
    horizontal_displacement = displacement_coefficient * d_mat / tmp_hyp
    
    # vertical displacement
    vertical_displacement = displacement_coefficient * depth / tmp_hyp
    
    # azimuthal angle
    azimuth = np.arctan2((easting-e1), (northing-n1))
    
    # compute north and east displacement from horizontal displacement and azimuth angle
    east_displacement = np.sin(azimuth) * horizontal_displacement
    north_displacement = np.cos(azimuth) * horizontal_displacement
    
    # project displacement field onto look vector
    temp = np.concatenate((east_displacement, north_displacement, vertical_displacement), axis=1)
    delta_range = temp.dot(np.transpose([plook]))
    delta_range = -1.0 * delta_range
    return delta_range

<font face="Calibri" size="3"><b>Write a function to create simulated deformation data based on Mogi Source Model parameters:</b> </font>

In [ ]:
def deformation_data_from_mogi(x, y, z, volume, iplot, imask):
    # Organizing model parameters
    bvc = [x, y, z, volume, 0, 0, 0, 0]
    bvc = np.transpose(bvc)
    
    # Setting acquisition parameters
    track =  -13.3*np.pi / 180.0
    look  = 23.0*np.pi / 180.0
    plook = [-np.sin(look)*np.cos(track), np.sin(look)*np.sin(track), np.cos(look)]
    
    # Defining easting and northing vectors
    northing = np.arange(0, (LINE)*POSTING, POSTING) / 1000
    easting = np.arange(0, (SAMPLE)*POSTING, POSTING) / 1000
    northing_mat = np.tile(northing, (SAMPLE, 1))
    easting_mat = np.transpose(np.tile(easting, (LINE, 1)))
    northing_vec = np.reshape(northing_mat, (LINE*SAMPLE, 1))
    easting_vec = np.reshape(easting_mat, (LINE*SAMPLE, 1))
    
    # Handing coordinates and model parameters over to the rngchg_mogi function
    calc_range = calc_forward_model_mogi(bvc[1], bvc[0], bvc[2], bvc[3], northing_vec, easting_vec, plook)
    
    # Reshaping surface deformation data derived via calc_forward_model_mogi()
    surface_deformation = np.reshape(calc_range, (SAMPLE,LINE))
    
    # return rotated surface deformation
    return np.transpose(np.fliplr(surface_deformation))

<hr>
<font face="Calibri" size="4"> <b> 3.4 Plotting The Mogi Forward Model</b><br></font>
<font face="Calibri" size="3">The cell below plots several Mogi forward models by varying some of the four main Mogi modeling parameters $(x_s,y_s,z_s,V)$.
    
The examples below fix the <i>depth</i> parameter to $z_s = 2.58 km$ and the <i>volume</i> change parameter to $volume = 0.0034 km^3$. We then vary the <i>easting</i> and <i>northing</i> parameters $x_s$ and $y_s$ to demonstrate how the model predictions vary when model parameters are changed.</font>
<br><br>
<font face="Calibri" size="3"><b>Run the first example:</b> </font>

In [ ]:
plt.rcParams.update({'font.size': 14})
extent_x = extents((np.arange(1, SAMPLE*POSTING, POSTING))/1000)
extent_y = extents((np.arange(1, LINE*POSTING, POSTING))/1000)
extent_xy = extent_x + extent_y
xs = np.arange(18, 24.2, 0.4)
ys = np.arange(20, 24.2, 0.4)

zs = 2.58;
volume = 0.0034;
xa = [0, 7, 15]
ya = [0 ,5, 10]

fig = plt.figure(figsize=(18, 18))
cmap = plt.cm.jet
subplot_index = 1

for k in xa:
    for l in ya: 
        ax = fig.add_subplot(3, 3, subplot_index)
        predicted_deformation_map = deformation_data_from_mogi(xs[k], ys[l], zs, volume, 0, 0)
        predicted_deformation_map_m = np.ma.masked_where(observed_deformation_map==0, predicted_deformation_map)
        im = ax.imshow(predicted_deformation_map_m, cmap=cmap, extent=extent_xy)
        cbar = ax.figure.colorbar(im, ax=ax, orientation='horizontal')
        plt.grid()
        im.set_clim(-30, 30)
        ax.plot(xs[k],ys[l], 'k*', markersize=25, markerfacecolor='w')
        ax.set_title('Source: X=%4.2fkm; Y=%4.2fkm' % (xs[k], ys[l]))
        ax.set_xlabel("Easting [km]")
        ax.set_ylabel("Northing [km]")
        subplot_index += 1
        
plt.savefig('Model-samples-3by3.png', dpi=200, transparent='false')

<br>
<hr>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #1</u>:  </font> Experiment with the Mogi Forward Model </b> <font color='rgba(200,0,0,0.2)'> -- [8 Points] </font> </font>

<font face="Calibri" size="3"> To get a feeling for the Mogi forward model, please run the following forward model experiments using the Python Function <i>deformation_data_from_mogi</i> and plot the results (using the code cell above):

<ol type="a">
<li>Run a reference simulation using the code cell above by specifying the following model parameters for source depth $x_s$ and volume change $V$: $z_{s1} = 2.5 km$; $V_1 = 0.01 km^3$. The script will visualize the resulting simulated surface deformation maps. Change the name of the output figure (last line of the script) to something that you will recognize later on (e.g., ReferenceRun.png).<font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li>
<br>
<li>Change the depth of the source by a factor of three ($z_{s2} = 7.5 km$) while leaving the other model parameters unchanged. Modify name of the output figure in the last line of the script. Visualize the results. Discuss changes to the reference run. Describe how the strength and shape of the deformation signal has changed and provide a physical explanation.<font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li>
<br>
<li>Now change the source volume by a factor of three ($V_2 = 0.03 km^3$ – also reset source depth to $z_{s1} = 2.5 km$). Visualize the results and compare them to the reference run.<font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li>
<br>
<li>Finally change both source volume and depth by a factor of three ($z_{s2} = 7.5 km$ and $V_2 = 0.03 km^3$). Compare this result to the results of experiments 1–3.<font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li>
</ol>

</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 1.1 [2 Points]:</font> Experiment no. 1: Perform reference run in code cell above using source model parameters to $z_{s1} = 2.5 km$; $V_1 = 0.01 km^3$. Plot results.</i> 

ADD DISCUSSION HERE:
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 1.2 [2 Points]:</font> Experiment no. 2: Set source model parameters to $z_{s2} = 7.5 km$; $V_1 = 0.01 km^3$. Plot and discuss the results in comparison to reference run.</i> 

ADD DISCUSSION HERE:
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 1.3 [2 Points]:</font> Experiment no. 3: Set source model parameters to $V_2 = 0.03 km^3$ and $z_{s1} = 2.5 km$. Plot and discuss the results in comparison to reference run.</i> 

ADD DISCUSSION HERE:
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 1.4 [2 Points]:</font> Experiment no. 4: Set source model parameters to $V_2 = 0.03 km^3$ and $z_{s2} = 7.5 km$. Plot and discuss the results in comparison to reference run.</i> 

ADD DISCUSSION HERE:
</font>
</div>
<hr>

<font face="Calibri" size="3"><b>Modify the example script below to answer questions 1.2 - 1.4:</b> </font>

In [ ]:
plt.rcParams.update({'font.size': 14})
extent_x = extents((np.arange(1, SAMPLE*POSTING, POSTING))/1000)
extent_y = extents((np.arange(1, LINE*POSTING, POSTING))/1000)
extent_xy = extent_x + extent_y
xs = np.arange(18, 24.2, 0.4)
ys = np.arange(20, 24.2, 0.4)

# ------------ Change Variables HERE --------------- #
zs = 2.58;
volume = 0.0034;
# ------------------------------------------------- #

xa = [0, 7, 15]
ya = [0 ,5, 10]

fig = plt.figure(figsize=(18, 18))
cmap = plt.cm.jet
subplot_index = 1

for k in xa:
    for l in ya: 
        ax = fig.add_subplot(3, 3, subplot_index)
        predicted_deformation_map = deformation_data_from_mogi(xs[k], ys[l], zs, volume, 0, 0)
        predicted_deformation_map_m = np.ma.masked_where(observed_deformation_map==0, predicted_deformation_map)
        im = ax.imshow(predicted_deformation_map_m, cmap=cmap,extent=extent_xy)
        cbar = ax.figure.colorbar(im, ax=ax, orientation ='horizontal')
        plt.grid()
        im.set_clim(-30, 30)
        ax.plot(xs[k],ys[l], 'k*', markersize=25, markerfacecolor='w')
        ax.set_title(f"Source: X={xs[k]:.2f}km; Y={ys[l]:.2f}km")
        ax.set_xlabel("Easting [km]")
        ax.set_ylabel("Northing [km]")
        subplot_index += 1

# CHANGE THE NAME OF THE IMAGE THAT IS BEING SAVED TO YOUR CLOUD INSTANCE!!
plt.savefig('Model-samples-3by3.png', dpi=200, transparent='false')

<hr>
<font face="Calibri" size="5"> <b> 4. Solving the [Pseudo]-Inverse Model</b><br></font>
<font face="Calibri" size="3"> The inverse problem seeks to determine the optimal parameters $(\hat{x_s},\hat{y_s},\hat{z_s},\hat{V})$ of the Mogi model $m$ by minimizing the <i>misfit</i> between predictions, $g(m)$, and observations $d^{obs}$ according to
    
\begin{equation}
\sum{\Big[g(m) - d^{obs}\Big]^2}
\end{equation}

This equation describes misfit using the <i>method of least-squares</i>, a standard approach to approximate the solution of an overdetermined equation system. We will use a <i>grid-search</i> approach to find the set of model parameters that minimize the the misfit function. The approach is composed of the following processing steps: 
<ol>
<li>Loop through the mogi model parameters,</li> 
<li>Calculate the forward model for each set of parameters,</li> 
<li>Calculate the misfit $\sum{[g(m) - d^{obs}]^2}$, and</li> 
<li>Find the parameter set that minimizes this misfit.</li>
</ol>
</font>

<hr>
<font face="Calibri" size="4"> <b> 4.1 Experimenting with Misfit</b></font>
<br><br>
<font face="Calibri" size="3">Let's <b>look at the misfit $\sum{[g(m) - d^{obs}]^2}$ for a number of different model parameter sets $(x_s,y_s,z_s,V)$:</b> 

</font>

In [ ]:
plt.rcParams.update({'font.size': 14})
extent_x = extents((np.arange(1, SAMPLE*POSTING, POSTING))/1000)
extent_y = extents((np.arange(1, LINE*POSTING, POSTING))/1000)
extent_xy = extent_x + extent_y
xs = np.arange(18, 24.2, 0.4)
ys = np.arange(20, 24.2, 0.4)

zs = 2.58;
volume = 0.0034;
xa = [0, 7, 15]
ya = [0 ,5, 10]

fig = plt.figure(figsize=(18, 18))
cmap = plt.cm.jet
subplot_index = 1

for k in xa:
    for l in ya: 
        ax = fig.add_subplot(3, 3, subplot_index)
        predicted_deformation_map = deformation_data_from_mogi(xs[k], ys[l], zs, volume, 0, 0)
        predicted_deformation_map_m = np.ma.masked_where(observed_deformation_map==0, predicted_deformation_map)
        im = ax.imshow(observed_deformation_map_m-predicted_deformation_map_m, cmap=cmap, extent=extent_xy)
        cbar = ax.figure.colorbar(im, ax=ax, orientation='horizontal')
        plt.grid()
        im.set_clim(-30, 30)
        ax.plot(xs[k], ys[l], 'k*', markersize=25, markerfacecolor='w')
        ax.set_title('Source: X=%4.2fkm; Y=%4.2fkm' % (xs[k], ys[l]))
        ax.set_xlabel("Easting [km]")
        ax.set_ylabel("Northing [km]")
        subplot_index += 1
plt.savefig('Misfit-samples-3by3.png', dpi=200, transparent='false')

<hr>
<font face="Calibri" size="4"> <b> 4.2 Running Grid-Search to Find Best Fitting Model Parameters $(\hat{x}_s,\hat{y}_s)$</b><br></font>
<font face="Calibri" size="3">The following code cell runs a grid-search approach to find the best fitting Mogi source parameters for the 2000-2002 deformation event at Okmok. To keep things simple, we will fix the depth $z_s$ and volume change $V$ parameters close to their "true" values and search only for the correct east/north source location ($x_s,y_s$).</font>
<br><br>
<font face="Calibri" size="3"><b>Write a script using the grid-search approach in Python:</b></font>

In [ ]:
# FIX Z AND dV, SEARCH OVER X AND Y

# Setting up search parameters
xs = np.arange(19, 22.2, 0.2)
ys = np.arange(21, 23.2, 0.2)
zs = 2.58;
volume = 0.0034;

nx = xs.size
ny = ys.size
ng = nx * ny;

print(f"fixed z = {zs}km, dV = {volume}, searching over (x,y)")

misfit = np.zeros((nx, ny))
subplot_index = 0

# Commence grid-search for best model parameters
for k, xv in enumerate(xs):
    for l, yv in enumerate(ys):
        subplot_index += 1
        predicted_deformation_map = deformation_data_from_mogi(xs[k], ys[l], zs, volume, 0, 0)
        predicted_deformation_map_m = np.ma.masked_where(observed_deformation_map==0, predicted_deformation_map)
        misfit[k,l] = np.sum(np.square(observed_deformation_map_m - predicted_deformation_map_m))
    print(f"Source {subplot_index:3d}/{ng:3d} is x = {xs[k]:.2f} km, y = {ys[l]:.2f} km")

# Searching for the minimum in the misfit matrix
mmf = np.where(misfit == np.min(misfit))
print(f"\n----------------------------------------------------------------")
print('Best fitting Mogi Source located at: X = %5.2f km; Y = %5.2f km' % (xs[mmf[0]], ys[mmf[1]]))
print(f"----------------------------------------------------------------")

<hr>
<font face="Calibri" size="4"> <b> 4.3 Plot and Inspect the Misfit Function</b><br></font>
<font face="Calibri" size="3">The code cell below plots the misfit function ($\sum{[g(m) - d^{obs}]^2}$) describing the fit of different Mogi source parameterizations to the observed InSAR data. You should notice a clear minimum in the misfit plot at the location of the best fitting source location estimated above. 
    
You may notice that, even for the best fitting solution, the misfit does not become zero. This could be due to other signals in the InSAR data (e.g., atmospheric effects or residual topography). Alternatively, it could also indicate that the observed deformation doesn't fully comply with Mogi theory. 
</font>
<br><br>
<font face="Calibri" size="3"><b>Plot the misfit function ($\sum{[g(m) - d^{obs}]^2}$):</b></font>

In [ ]:
plt.rcParams.update({'font.size': 18})
extent_xy = extents(xs) + extents(ys)
fig = plt.figure(figsize=(10, 10))
cmap = plt.cm.jet
ax1 = fig.add_subplot(1, 1 ,1)
im = ax1.imshow(np.transpose(misfit), origin='lower', cmap=cmap, extent=extent_xy)
# USE THIS COMMAND TO CHANGE COLOR SCALING: im.set_clim(-30, 30)
ax1.set_aspect('auto')
cbar = ax1.figure.colorbar(im, ax=ax1, orientation='horizontal')
ax1.plot(xs[mmf[0]], ys[mmf[1]], 'k*', markersize=25, markerfacecolor='w')
ax1.set_title("Misfit Function for Mogi-Source Approximation")
ax1.set_xlabel("Easting [km]")
ax1.set_ylabel("Northing [km]")
plt.savefig('Misfit-function.png', dpi=200, transparent='false')

<hr>
<font face="Calibri" size="4"> <b> 4.4 Plot Best-Fitting Mogi Forward Model and Compare to Observations</b><br></font>
<font face="Calibri" size="3">With the best-fitting model parameters defined, you can now analyze how well the model fits the InSAR-observed surface deformation. The best way to do that is to look at both the observed and predicted deformation maps and compare their spatial patterns. Additionally, we will also plot the residuals (<i>observed_deformation_map</i> - <i>observed_deformation_map</i>) to determine if there are additional signals in the data that are not modeled using Mogi theory. 
</font>
<br><br>
<font face="Calibri" size="3"><b>Compare the observed and predicted deformation maps:</b></font>

In [ ]:
# Calculate predicted deformation map for best-fitting Mogi parameters:
predicted_deformation_map = deformation_data_from_mogi(xs[mmf[0]], ys[mmf[1]], zs, volume, 0, 0)

# Mask the predicted deformation map to remove pixels incoherent in the observations:
predicted_deformation_map_m = np.ma.masked_where(observed_deformation_map==0, predicted_deformation_map)

# Plot observed deformation map
plot_model(observed_deformation_map_m, LINE, SAMPLE, POSTING)

# Plot simulated deformation map
plot_model(predicted_deformation_map_m, LINE, SAMPLE, POSTING)

plt.savefig('BestFittingMogiDefo.png', dpi=200, transparent='false')

# Plot simulated deformation map without mask applied
plot_model(predicted_deformation_map, LINE, SAMPLE, POSTING)

<font face="Calibri" size="3"><b>Determine if there are additional signals in the data that are not modeled using Mogi theory:</b></font>

In [ ]:
# Plot residual between observed and predicted deformation maps
plot_model(observed_deformation_map_m-predicted_deformation_map_m, LINE, SAMPLE, POSTING)
plt.savefig('Residuals-ObsMinusMogi.png', dpi=200, transparent='false')

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #2</u>:  </font> Run 2nd Grid-Search to Find Model Parameters $(\hat{z}_s,\hat{V})$  </b> <font color='rgba(200,0,0,0.2)'> -- [8 Points] </font> </font>

<font face="Calibri" size="3"> For this second grid-search run, we now switch out the model parameters we are trying to estimate. We will assume that the lateral location of the Mogi source is now fixed to its estimated value ($\hat{x}_s = 20.6 km$; $\hat{y}_s = 21.8 km$). 

<u>To perform a grid search for the best fitting model parameters $\hat{z}_s$ and $\hat{V}$this exercise, please complete the following steps</u>:

<ol>
<br>
<li>Using the previous grid-search script as a template, <b>write a new grid-search script to search for the best fitting source model depth ($z_s$) and volume change ($V$)</b>.<font color='rgba(200,0,0,0.2)'> -- [3 Points] </font></li>
<br>
<li>Provide a plot of the misfit function and provide the best-fitting values for $\hat{z}_s$ and $\hat{V}$. When plotting the misfit function, put $z_s$ on the vertical axis. You may want to adjust the color scale, in order to better see the shape of the misfit function.<font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li>
<br>
<li>Compare the $z_s$ vs $V$ misfit function (misfit function 2) to the $y_s$ vs $x_s$ misfit function (misfit function 1). You should see that the shape of the function is different. Misfit function 1 is largely of circular shape while misfit function 2 appears elongated. Interpret this pattern.<font color='rgba(200,0,0,0.2)'> -- [3 Points] </font></li>
</ol>

</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 2.1 [3 Points]:</font> Provide code to perform grid search over $z_s$ and $V$.</i> 

<b>PROVIDE SCRIPT BY MODIFYING THE CODE IN THE CODE CELL BELOW:</b>
</font>
</div>

In [ ]:
# !!!! MODIFY THIS SCRIPT TO PERFORM A GRID SEARCH OVER zs AND V: !!!!

# Setting up search parameters
xs = np.arange(19, 22.2, 0.2)
ys = np.arange(21, 23.2, 0.2)
zs = 2.58;
volume = 0.0034;

nx = xs.size
ny = ys.size
ng = nx * ny;

#print('fixed z = ',zs,' km, dV = ',volume, ' searching over (x,y)')
print(f"fixed z = {zs}km, dV = {volume}, searching over (x,y)")

misfit=np.zeros((nx,ny))
subplot_index = 0

# Commence grid-search for best model parameters
for k, xv in enumerate(xs):
    for l, yv in enumerate(ys):
        subplot_index = subplot_index+1
        predicted_deformation_map = deformation_data_from_mogi(xs[k],ys[l],zs,volume,0,0)
        predicted_deformation_map_m = np.ma.masked_where(observed_deformation_map == 0, predicted_deformation_map)
        misfit[k,l] = np.sum(np.square(observed_deformation_map_m - predicted_deformation_map_m))
    print(f"Source {subplot_index:3d}/{ng:3d} is x = {xs[k]:.2f} km, y = {ys[l]:.2f} km")

# Searching for the minimum in the misfit matrix
mmf = np.where(misfit == np.min(misfit))
print('')
print(f"\n----------------------------------------------------------------")
print('Best fitting Mogi Source located at: X = %5.2f km; Y = %5.2f km' % (xs[mmf[0]], ys[mmf[1]]))
print(f"----------------------------------------------------------------")

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 2.2-A [1 Points]:</font> Provide the best fitting values for source depth ($\hat{x}_s$) and volume change ($\hat{V}$) according to your grid-search results.</i> 

PROVIDE ESTIMATES FOR $\hat{x}_s$ and $\hat{V}$ HERE:
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 2.2-B [1 Points]:</font> Provide plot of $x_s$ on $V$ misfit function.</i> 

PROVIDE PLOT HERE:
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 2.3 [3 Points]:</font> Compare the $z_s$ vs $V$ misfit function (misfit function 2) to the initial $y_s$ vs $x_s$ misfit function (misfit function 1). Interpret their difference in spatial pattern.</i> 

PROVIDE DISCUSSION HERE:
</font>
</div>

<br>
<hr>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #3</u>:  </font> Error Discussion </b> <font color='rgba(200,0,0,0.2)'> -- [4 Points] </font> </font>

<font face="Calibri" size="3"> In a perfect world were data are noise-free geophysical models perfectly represent reality, there should be a set of model parameters that reduces the misfit function to zero. In our case, however, the misfit function still shows large values, even for the best fitting model parameters. Provide and explain three reasons for why the differences between the model and the data are non-zero.
<br><br>
PROVIDE DISCUSSION HERE:
</font>
</div>
<hr>

<font face="Calibri" size="2"> <i>GEOS 657 Microwave Remote Sensing - Version 1.0 - March 2019 </i>
</font>